### Look into adding control non-inflammtories from UCSF to the "controls" group

In [39]:
library(tidyverse)
library(data.table)
library(ggplot2)
library(palettetown)
library(stringr)
library(dplyr)
library(ggpattern)
source('../0_support-files/theme_CRP-MISC.R')
source('../0_support-files/prevail_man_fill.R')
library(ggpubr)
library(rstatix)




expGroupPalette <- c("Control" = "#00FF7F",
                            "COVID-19_Nonsevere" = "#7DB27D",
                            "COVID-19_Severe" = "#7DB27D",
                            "Control Non-inflammatory" ='#FBE77C',
                            "Acute COVID-19 Mild/ASX" = '#F0484E',
                            "Acute COVID-19 Moderate/Severe" = '#F0484E',
                            "Acute MIS-C Moderate/Severe" = '#5CB2EB')



'%ni%' <- Negate('%in%')

In [57]:
solid_organ = c("colon","heart","kidney","liver","lung","pancreas","skin","spleen")


expGroupPalette <- c("Control" = "#00FF7F",
                            "COVID-19_Nonsevere" = "#7DB27D",
                            "COVID-19_Severe" = "#7DB27D",
                            "Control_Non-inflammatory" ='#FBE77C',
                            "Acute COVID-19 Mild/ASX" = '#F0484E',
                            "Acute COVID-19 Moderate/Severe" = '#F0484E',
                            "Acute MIS-C Moderate/Severe" = '#5CB2EB')


#------------------------------
## Read in MISC data
master_df = read.csv("../1_sample-data/STable8_cfdna-samples.csv") %>% 
    rename(scaled_cfDNA = cfDNA_concentration) %>%
    mutate(severity = ifelse(severity == 0 | severity == 1,"Mild/ASX",
                             ifelse(severity == 2 | severity == 3,"Moderate/Severe",""))) %>%
    mutate(Diagnosis = ifelse(Diagnosis != "Control_Non-inflammatory", paste0("Acute ",Diagnosis," ",severity),Diagnosis)) %>%
    rename(Sample_ID = cfdna_sample_id) %>% 
    select(Sample_ID,PTID,Diagnosis,scaled_cfDNA,all_of(solid_organ),severity)

#------------------------------
## Read in adult cohort
adult_mdf <- read.delim("../1_sample-data/cfdna_master.adult.tsv") %>%
    filter(origin == "MCGILL") %>%
    mutate(Diagnosis = severity) %>%
    mutate(expGroup_plotting = Diagnosis) %>%
    mutate(icu = 0) %>%
    select(Sample_ID,PTID,Diagnosis,scaled_cfDNA,all_of(solid_organ),severity)

# head(adult_mdf)

#------------------------------
## Combine
all_abd = rbind(adult_mdf,master_df)

#------------------------------
## Filter and Re-factor


all_abd$Diagnosis <- factor(all_abd$Diagnosis, levels=names(expGroupPalette))

# all_abd$icu <- factor(all_abd$icu)


# ## Add solid tissue part

all_abd$solid_organ_frac <- rowSums(all_abd[,colnames(all_abd) %in% solid_organ])
all_abd$solid_organ_conc <- all_abd$solid_organ_frac * all_abd$scaled_cfDNA

all_abd$expGroup_plotting <- all_abd$Diagnosis
# all_abd$expGroup_plotting <- factor(all_abd$expGroup_plotting, levels=names(expGroupPalette))



---
# FOR PUB

In [58]:
all_abd <- all_abd %>% select(expGroup_plotting,Sample_ID,solid_organ_conc,scaled_cfDNA) %>% reshape2::melt(id.vars= c("expGroup_plotting","Sample_ID"))

all_abd$expGroup_plotting <- factor(all_abd$expGroup_plotting, levels = names(expGroupPalette))
head(all_abd)

,expGroup_plotting,Sample_ID,variable,value
,<fct>,<chr>,<fct>,<dbl>
1,COVID-19_Nonsevere,MCGILL1,solid_organ_conc,0.005827706
2,COVID-19_Nonsevere,MCGILL20,solid_organ_conc,0.072144049
3,COVID-19_Nonsevere,MCGILL21,solid_organ_conc,0.026977047
4,COVID-19_Nonsevere,MCGILL25,solid_organ_conc,0.008103787
5,COVID-19_Nonsevere,MCGILL27,solid_organ_conc,0.006719510
6,COVID-19_Nonsevere,MCGILL26,solid_organ_conc,0.007427791


In [60]:
pdf(file="plots/panelB.pdf",
                 width=3.47,height=2.25, paper="special", bg="white",
                 fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

# XORDER <- c('Control \nn=4','COVID-19 \nNonsevere \nn=30','COVID-19 \nSevere \nn=22',
#             'Control \nn=5','COVID-19 \n acute \nn=18','MIS-C \n acute \nn=40')

YLIM = 10

final_plt <- all_abd %>% 
# mutate(Diagnosis = factor(Diagnosis,levels = XORDER)) %>%
ggplot(aes(x=expGroup_plotting,
           y=value,
           fill=expGroup_plotting,
           pattern=variable))+ #fill=Diagnosis,
    geom_boxplot(outlier.shape = NA,
                 size = 0.1)+                     # place holder to set discrete scale
    theme_prevail()+
    theme(plot.margin = unit(c(2,2,2,2), "pt"),
            axis.title.x = element_blank())+
    # scale_fill_poke(pokemon = 137, spread = 6)+
#     coord_cartesian(ylim = c(0,YLIM))+
    annotate(geom="rect",
             xmin = 3.5,
             xmax = Inf, 
             ymin = 0, 
             ymax = Inf, 
             alpha = .2)+              # add rect
    geom_boxplot_pattern(outlier.shape = NA,
                         size = 0.2,
                         position = position_dodge(preserve = "single"),
                         color = "black",
                         pattern_size=0.2,
                         pattern_colour = "black",
                         pattern_fill = "white",
                         pattern_angle = 45,
                         pattern_density = 0.1,
                         pattern_spacing = 0.025,
                         pattern_key_scale_factor = 0.6) +
    geom_point(position =  position_jitterdodge(jitter.height=0, 
                                                jitter.width=.25, 
                                                seed=42),
                size = 0.4)+
    scale_fill_manual(values = expGroupPalette) +
    scale_pattern_manual(values = c(solid_organ_conc = "stripe", 
                                    scaled_cfDNA = "none"))+
    coord_cartesian(ylim = c(0,YLIM))+
    scale_y_continuous(breaks = c(2,4,6,8))

##---------------------------------------
# ADD ARROWS

subset <- names(expGroupPalette)

mapping_key <- c(1:length(subset))
names(mapping_key) <- subset

outliers <- all_abd %>% filter(value > YLIM)

text_offset = 0.09
text_size = 2.25
text_lineheight = .75

arrow_size = 0.5

for (i  in 1:length(subset)){
    group <- subset[i]
    group_outliers <- outliers %>% filter(expGroup_plotting == group)
    
    ## ADD ARROW
    if (nrow(group_outliers) > 0){
        
        outlier_values <- round(group_outliers$value,2)
        outlier_values <- as.character(outlier_values[order(outlier_values,decreasing=TRUE)])
                
        final_plt <- final_plt + annotate("segment", x = i, xend = i,
                                          y = 7.5, yend = 8,
                                          size = arrow_size, lineend="butt", linejoin="mitre", arrow=arrow(length=unit(.02,"npc")))

        final_plt <- final_plt + annotate("text", x = i+text_offset, y = YLIM, hjust=0,vjust=1.2,
                                          label = paste0(outlier_values,collapse="\n"),
                                          size = text_size, family = "Helvetica", lineheight = text_lineheight)

        
    }
}

print(final_plt)
dev.off()

png 
  2